In [1]:
import pandas as pd
import numpy as np

CBOE Data ------------------------------------------------------------------------------------------------------------

In [2]:
# Load in data
vix = pd.read_csv("CBOE/VIX.csv")
vox = pd.read_csv("CBOE/VOX.csv")
vxd = pd.read_csv("CBOE/VXD.csv")
vxn = pd.read_csv("CBOE/VXN.csv")

In [3]:
# Rename columns
vox.columns = ['Date', 'VOX Open', 'VOX High', 'VOX Low', 'VOX Close']
vxd.columns = ['Date', 'VXD Open', 'VXD High', 'VXD Low', 'VXD Close']
vxn.columns = ['Date', 'VXN Open', 'VXN High', 'VXN Low', 'VXN Close']

# Can comment out if we need other columns
# Only take the close columns
vix = vix[['Date', 'VIX Close']]
vox = vox[['Date', 'VOX Close']]
vxd = vxd[['Date', 'VXD Close']]
vxn = vxn[['Date', 'VXN Close']]

In [4]:
# Drop N/A dates
vix = vix[:7236] # These dates are in the future so we don't need them

In [5]:
# Fix dates (only VIX needs to be fixed)
dates = []
for date in vix['Date']:
    parts = date.split('/')
    month = parts[0]
    if(month[0] == '0'):
        month = month[1]
    day = parts[1]
    if(day[0] == '0'):
        day = day[1]
    year = parts[2]
    dates.append(month+'/'+day+'/'+year)

vix['Date'] = dates

In [6]:
# Join them
temp = pd.merge(vix, vox, on='Date', how='outer')
temp2 = pd.merge(temp, vxd, on='Date', how='outer')
CBOE = pd.merge(temp2, vxn, on='Date', how='outer')

CRSP Data ------------------------------------------------------------------------------------------------------------

In [7]:
# Load in data
meta = pd.read_csv("CRSP/CRSP_CAP_metadata.csv")
cap = pd.read_csv("CRSP/CRSP_CAP.csv")

In [8]:
# Remove unnecessary columns
cap = cap[cap.columns[0:20]]

cols = []
cols.append('Date')
cols.extend(cap.columns[1:])
cap.columns = cols

In [9]:
# Fix date format
dates = []
for date in cap['Date']:
    year = str(date)[2:4]
    month = str(date)[4:6]
    if(month[0] == '0'):
        month = month[1]
    day = str(date)[6:8]
    if(day[0] == '0'):
        day = day[1]
    dates.append(month+'/'+day+'/'+year)

cap['Date'] = dates

In [10]:
# Drop N/A values and Fix column names
#cap = cap.dropna()
temp = [cap.columns[0]]
temp.extend(meta.columns[1:])
cap.columns = temp

SP500 Data -----------------------------------------------------------------------------------------------------------

In [11]:
sp500 = pd.read_csv("SP500/sp500.csv")
# Rename columns
sp500.columns = ['Date', 'sp500', 'log_sp500']

In [23]:
temp1 = pd.merge(CBOE, cap, on='Date', how='outer')
temp2 = pd.merge(temp1, sp500, on='Date', how="outer")

In [24]:
temp2.dropna().shape

(4376, 26)

FRED Data ------------------------------------------------------------------------------------------------------------

In [34]:
# import FRED data
FRED = pd.read_csv("FRED_Merged.csv")

In [35]:
temp = FRED[FRED.columns[1:]]
result = pd.merge(temp2, temp, on='Date', how='outer')

French Data ----------------------------------------------------------------------------------------------------------

In [48]:
# import French data
French = pd.read_csv("French_data.csv")

In [49]:
French = French.drop('Unnamed: 0', axis=1)

In [51]:
Final = pd.merge(result, French, on='Date', how='outer')

In [54]:
Final.to_csv("Total_Data.csv")